In [1]:
import pandas as pd
from fuzzywuzzy import fuzz, process

In [2]:
phii = pd.read_excel(r'G:/My Drive/Spatial data/DED/Klaster PHII.xlsx')

geom = pd.read_csv(r'G:/My Drive/Spatial data/Superset/geom_desa_garut_20240610.csv')
adm = pd.read_excel(r'G:/My Drive/Spatial data/Nama Lokasi/Garut - Daerah Administratif.xlsx')
phii = phii[phii['Kabupaten']=='GARUT']

# geom = pd.read_csv(r'G:/My Drive/Spatial data/Superset/geom_desa_badung_20240610.csv')
# adm = pd.read_excel(r'G:/My Drive/Spatial data/Nama Lokasi/Badung - Daerah Administratif.xlsx')
# phii = phii[phii['Kabupaten']=='BADUNG']

In [3]:
def fuzzy_match(row, choices, scorer=fuzz.ratio, cutoff=0):
    """
    Perform fuzzy matching on the row and return the best match and its score.
    
    :param row: The row to match
    :param choices: List of choices to match against
    :param scorer: Scoring function from fuzzywuzzy
    :param cutoff: Minimum score to be considered a match
    :return: Best match and its score
    """
    match, score = process.extractOne(row, choices, scorer=scorer, score_cutoff=cutoff)
    return pd.Series([match, score])

In [4]:
# Convert the 'ID_fasilitas_kesehatan' column in the 'phii' DataFrame to string type,
# split the string on '.' and take the first part of the split result.
phii['ID_fasilitas_kesehatan'] = phii['ID_fasilitas_kesehatan'].astype(str).str.split('.').str[0]

# Remove the substring 'PUSKESMAS' from the 'nama_fasilitas_kesehatan' column in 'phii' DataFrame
# and strip any leading or trailing whitespace.
phii['nama_fasilitas_kesehatan'] = phii['nama_fasilitas_kesehatan'].str.replace('PUSKESMAS', '', regex=False).str.strip()

# Select the 'Desa/Village', 'Puskesmas', and 'Kecamatan/District' columns from the 'adm' DataFrame
# and drop any rows with missing values.
adm = adm[['Desa/Village', 'Puskesmas', 'Kecamatan/District']].dropna()

# Apply a fuzzy matching function to 'nama_fasilitas_kesehatan' in 'phii' to match with 'Puskesmas' in 'adm',
# storing the best match and the matching score.
phii[['best_match', 'score']] = phii['nama_fasilitas_kesehatan'].apply(fuzzy_match, choices=adm['Puskesmas'].unique())

# Filter 'phii' to keep only the rows with a matching score of 90 or higher,
# then merge with 'adm' on the matched 'Puskesmas' name.
phii_select = phii[phii['score'] >= 90].merge(adm, left_on='best_match', right_on='Puskesmas')

# Create a new 'id' column in 'phii_select' by concatenating 'Kecamatan/District' and 'Desa/Village'.
phii_select['id'] = phii_select['Kecamatan/District'] + ' ' + phii_select['Desa/Village']

# Replace underscores with spaces in the 'NAMOBJ' column of 'geom' to create an 'id' column.
geom['id'] = geom['NAMOBJ'].str.replace('_', ' ')

# Apply a fuzzy matching function to 'id' in 'phii_select' to match with 'id' in 'geom',
# storing the best match and the matching score.
phii_select[['best_match', 'score']] = phii_select['id'].apply(fuzzy_match, choices=geom['id'].unique())

# Sort 'phii_select' by matching score, dropping duplicates based on 'best_match' and keeping the last occurrence.
phii_select = phii_select.sort_values('score').drop_duplicates('best_match', keep='last')

# Merge 'geom' with the relevant columns from 'phii_select' based on the 'id' in 'geom' and 'best_match' in 'phii_select'.
geom_phii = geom.merge(phii_select[['ID_fasilitas_kesehatan', 'nama_fasilitas_kesehatan', 'CLUSTER', 'best_match']], 
                       how='left', left_on='id', right_on='best_match')

# Select and reorder the columns in the 'geom_phii' DataFrame.
geom_phii = geom_phii[['NAMOBJ', 'WADMKD', 'WADMKC', 'WADMKK', 'ID_fasilitas_kesehatan', 'nama_fasilitas_kesehatan', 'CLUSTER', 'geometry']]

# Fill any missing values in the 'CLUSTER' column of 'geom_phii' with 0.
geom_phii['CLUSTER'] = geom_phii['CLUSTER'].fillna(0)